In [1]:
import pandas as pd
import json
import numpy as np
from collections import Counter
import re
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, StringType

### Prétraitement

In [2]:
# --- GÉNÉRATION DU FICHIER RÉPONSES (Questionnaire + Lieu) ---

# Chargement du CSV original
df_reponses = pd.read_csv('../data_coord/reponses-epci-200046977.csv', low_memory=False, quotechar='"')

# Sélection des colonnes stratégiques selon la notice [cite: 86, 88, 90]
cols_reponses = [
    'insee',        # Lieu (Code INSEE) [cite: 21, 29]
    'q6',           # Fréquence de pratique [cite: 86]
    'q14',          # Sentiment de sécurité [cite: 86]
    'q18',          # Dangerosité carrefours [cite: 88]
    'q21',          # État de l'entretien [cite: 88]
    'q28',          # Conflit stationnement motorisé [cite: 88]
    'q34_texte',    # Les 3 priorités d'amélioration [cite: 88]
    'q35',          # Commentaire libre (NLP) [cite: 88]
    'q38_texte',    # Situations de violence vécues [cite: 90]
    'score'         # Note globale de la commune [cite: 30]
]

# Filtrage et nettoyage
df_reponses_final = df_reponses[cols_reponses].copy()
df_reponses_final['q35'] = df_reponses_final['q35'].str.replace(r'\r+|\n+', ' ', regex=True).fillna('')

# Sauvegarde du second fichier
df_reponses_final.to_csv('../resultats_croisement_complet/reponses_questionnaire_lyon.csv', index=False)

print("Les deux fichiers ont été générés avec succès :")
print("- reponses_questionnaire_lyon.csv (Données thématiques et profils)")
print(df_reponses_final.shape)

Les deux fichiers ont été générés avec succès :
- reponses_questionnaire_lyon.csv (Données thématiques et profils)
(12567, 10)


In [3]:
# 1. INITIALISATION DE LA SESSION
spark = SparkSession.builder.appName("Datathon_Lyon_Complet").getOrCreate()

### NLP

In [4]:


# 2. CONFIGURATION RÉFÉRENTIELLE
CONFIG_PROBLEMES = {
    'Infrastructure manquante': {'mots': r'piste|manque|absence|discontinuité|coupure', 'urgence': 3, 'facilite': 3},
    'Carrefours dangereux': {'mots': r'carrefour|intersection|rond-point|traversée', 'urgence': 3, 'facilite': 3},
    'Vitesse excessive': {'mots': r'vitesse|rapide|trop vite|ralentir', 'urgence': 3, 'facilite': 2},
    'Violence routière': {'mots': r'violence|insulte|intimidation|agression', 'urgence': 3, 'facilite': 2},
    'Entretien pistes': {'mots': r'entretien|entretenir|nid de poule|trou|verre', 'urgence': 2, 'facilite': 1},
    'Stationnement gênant': {'mots': r'stationnement|garé|voiture|parking|double file', 'urgence': 2, 'facilite': 2},
    'Stationnement vélo': {'mots': r'stationnement vélo|parking vélo|garage vélo', 'urgence': 2, 'facilite': 1},
    'Limitation trafic': {'mots': r'limiter trafic|réduire trafic|moins de voiture', 'urgence': 2, 'facilite': 3},
    'Signalisation': {'mots': r'panneau|feu|signalisation|marquage', 'urgence': 1, 'facilite': 1},
    'Conflit piétons': {'mots': r'piéton|trottoir|partagé|quai', 'urgence': 1, 'facilite': 2},
}

def get_label_facilite(score):
    if score <= 1.5: return "🟢 Facile"
    if score <= 2.5: return "🟡 Moyen"
    return "🔴 Difficile"

# Partage de la config sur le cluster
conf_broadcast = spark.sparkContext.broadcast(CONFIG_PROBLEMES)

# 3. FONCTION DE DÉTECTION (UDF)
def detecter_themes_py(texte):
    if texte is None or texte == "": return []
    res = []
    for nom, cfg in conf_broadcast.value.items():
        if re.search(cfg['mots'], str(texte).lower()):
            res.append(nom)
    return res

detecter_themes_udf = F.udf(detecter_themes_py, ArrayType(StringType()))

# 4. CHARGEMENT ET NETTOYAGE ROBUSTE
print("Chargement des données...")
# On utilise multiLine et quote pour éviter que les commentaires ne cassent les colonnes
df_raw = spark.read.option("header", "true").option("multiLine", "true").option("quote", "\"").option("escape", "\"").csv('../resultats_croisement_complet/reponses_questionnaire_lyon.csv')

# Filtre : On ne garde que les codes INSEE valides (5 chiffres) pour éviter les décalages de texte
df_clean = df_raw.filter(F.col("insee").rlike("^[0-9]{5}$"))

# Cast de la sécurité (q14) : les textes deviennent NULL proprement
df_spark = df_clean.withColumn("q14", F.expr("try_cast(q14 as double)"))

# Fusion des textes pour analyse
df_processed = df_spark.withColumn(
    'verbatim', 
    F.concat_ws(' ', F.coalesce(F.col('q34_texte'), F.lit('')), 
                     F.coalesce(F.col('q35'), F.lit('')), 
                     F.coalesce(F.col('q38_texte'), F.lit('')))
).withColumn('problemes_list', detecter_themes_udf(F.col('verbatim')))

# 5. ANALYSE PAR TYPE DE PROBLÈME (GLOBAL)
print("\n ANALYSE DE LA FACILITÉ PAR TYPE DE PROBLÈME")
df_exploded = df_processed.select(F.explode("problemes_list").alias("probleme"))
counts_global = {row['probleme']: row['count'] for row in df_exploded.groupBy("probleme").count().collect()}

analyse_p = []
for nom, cfg in CONFIG_PROBLEMES.items():
    analyse_p.append({
        'Problème': nom,
        'Citations': counts_global.get(nom, 0),
        'Urgence': cfg['urgence'],
        'Facilité': cfg['facilite'],
        'Type': get_label_facilite(cfg['facilite'])
    })

df_facilite_theme = pd.DataFrame(analyse_p).sort_values(by='Facilité')
print(df_facilite_theme.to_string(index=False))

# 6. ANALYSE PAR COMMUNE (TOP 10)
print("\n ANALYSE PAR COMMUNE (TOP 10)")

# Sécurité Moyenne
df_secu = df_processed.groupBy("insee").agg(
    F.count("*").alias("Réponses"),
    F.round(F.avg("q14"), 2).alias("Sécurité_Moy")
)

# Calcul du Top 10 des thèmes par commune
df_exploded_commune = df_processed.select("insee", F.explode("problemes_list").alias("probleme"))
df_counts_commune = df_exploded_commune.groupBy("insee", "probleme").count()

window_spec = Window.partitionBy("insee").orderBy(F.col("count").desc())
df_ranked = df_counts_commune.withColumn("rank", F.row_number().over(window_spec)) \
    .filter(F.col("rank") <= 10) \
    .withColumn("p_fmt", F.concat(F.col("probleme"), F.lit(" ("), F.col("count"), F.lit(")")))

df_top10 = df_ranked.groupBy("insee").agg(F.concat_ws(" | ", F.collect_list("p_fmt")).alias("Top_10_Problemes"))

# 7. EXPORT FINAL
df_final = df_secu.join(df_top10, "insee", "left").orderBy(F.col("Réponses").desc())
result_pandas = df_final.toPandas()
result_pandas.to_csv("../resultats_croisement_complet/analyse_finale_lyon.csv", index=False)

print("\n" + "="*120)
print(result_pandas.head(15).to_string(index=False))
print("="*120)

Chargement des données...

 ANALYSE DE LA FACILITÉ PAR TYPE DE PROBLÈME
                Problème  Citations  Urgence  Facilité        Type
        Entretien pistes       5754        2         1    🟢 Facile
      Stationnement vélo         32        2         1    🟢 Facile
           Signalisation        465        1         1    🟢 Facile
       Vitesse excessive       9720        3         2     🟡 Moyen
       Violence routière       7171        3         2     🟡 Moyen
    Stationnement gênant       5199        2         2     🟡 Moyen
         Conflit piétons       6363        1         2     🟡 Moyen
Infrastructure manquante      10169        3         3 🔴 Difficile
    Carrefours dangereux        348        3         3 🔴 Difficile
       Limitation trafic         11        2         3 🔴 Difficile

 ANALYSE PAR COMMUNE (TOP 10)

insee  Réponses  Sécurité_Moy                                                                                                                                  